In [2]:
%pip install --upgrade fastf1


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import fastf1
import datetime

import pandas as pd
import numpy as np

from typing import List

In [4]:
datetime.datetime.today()

datetime.datetime(2024, 11, 11, 14, 46, 10, 548585)

In [242]:
def rank_quali_laps(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ranks lap times by speed within each QualiSession.
    The fastest lap gets rank 1, updating as new laps are set.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing qualifying data with LapTime and QualiSession columns
    
    Returns:
    pandas.DataFrame: Original DataFrame with new 'LapRank' and 'CurrentFastestLap' columns
    """
    
    # Sort by QualiSession and LapStartDate to maintain chronological order
    df = df.sort_values(["Event", "QualiSession", "LapStartDate"])
    
    def update_rankings(group):
        # Create a running ranking based on laptimes seen so far
        group = group.copy()
        for idx in range(len(group)):
            current_slice = group.iloc[:idx + 1]
            # Rank the laptimes
            current_ranks = current_slice["LapTime"].rank(method="min", ascending=True)
            group.iloc[idx, group.columns.get_loc("LapRank")] = current_ranks.iloc[-1]
            
            # Store the current fastest lap time for reference
            group.iloc[idx, group.columns.get_loc("CurrentFastestLap")] = \
                current_slice["LapTime"].min()
        
        return group
    
    # Initialize new columns
    df["LapRank"] = float("nan")
    df["CurrentFastestLap"] = pd.Timedelta(0)
    
    # Apply the ranking function to each QualiSession group
    df = df.groupby("QualiSession", group_keys=False).apply(update_rankings)

    # Calculate the delta between CurrentFastestLap and LapTime
    df["DeltaFastestLap"] = df["LapTime"] - df["CurrentFastestLap"]
    
    return df

In [248]:
def extract_data(
    year: List[int],
    session_type: str,
) -> pd.DataFrame:
    
    cols_to_keep = ["Event", "Driver", "Team", "LapTime", "LapNumber", "Stint", "Sector1Time", "Sector2Time", "Sector3Time", 
                "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "IsPersonalBest", "Compound", "TyreLife", "FreshTyre",
                "TrackStatus", "Deleted", "DeletedReason", "LapStartDate"]
    
    dfs = []
    for i in year:
        schedule = fastf1.get_event_schedule(i)
        gps = list(schedule[(schedule["EventFormat"]!="testing") & (schedule["EventDate"]<datetime.datetime.today())]["EventName"])

        for gp in gps:
            print(f"{i} {gp}")
            session = fastf1.get_session(i, gp, session_type)
            session.load()
            
            laps_df = session.laps
            laps_df["Event"] = f"{i} {gp}"

            laps_df = laps_df[(laps_df["PitInTime"].isna()) & (laps_df["PitOutTime"].isna())]

            laps_df = laps_df[cols_to_keep]

            race_control_df = session.race_control_messages
            quali_sessions = race_control_df[(race_control_df.Message.str.contains("GREEN LIGHT")) |
                                (race_control_df.Message.str.contains("CHEQUERED FLAG")) |
                                (race_control_df.Message.str.contains("WILL NOT BE RESUMED"))]

            # Get starting times for each qualifying session
            quali_sessions["GreenLightCount"] = (quali_sessions["Message"].str.contains("GREEN LIGHT")
                                                .groupby((quali_sessions["Message"]!=quali_sessions["Message"].shift()).cumsum())
                                                .cumsum())
            quali_sessions = quali_sessions[quali_sessions["GreenLightCount"]==1] # first instance of green light means start of session

            quali_sessions["QualiSession"] = ["Q" + str(i) for i in range(1, len(quali_sessions) + 1)]
            quali_sessions = quali_sessions[["Time", "QualiSession"]]

            q1_mask = (laps_df["LapStartDate"] >= quali_sessions.loc[quali_sessions["QualiSession"] == "Q1", "Time"].iloc[0]) & \
                        (laps_df["LapStartDate"] < quali_sessions.loc[quali_sessions["QualiSession"] == "Q2", "Time"].iloc[0])
                    
            q2_mask = (laps_df["LapStartDate"] >= quali_sessions.loc[quali_sessions["QualiSession"] == "Q2", "Time"].iloc[0]) & \
                        (laps_df["LapStartDate"] < quali_sessions.loc[quali_sessions["QualiSession"] == "Q3", "Time"].iloc[0])
                
            q3_mask = laps_df["LapStartDate"] >= quali_sessions.loc[quali_sessions["QualiSession"] == "Q3", "Time"].iloc[0]

            laps_df.loc[q1_mask, "QualiSession"] = "Q1"
            laps_df.loc[q2_mask, "QualiSession"] = "Q2"
            laps_df.loc[q3_mask, "QualiSession"] = "Q3"

            quali_df = rank_quali_laps(laps_df)

            dfs.append(quali_df)

    return pd.concat(dfs)

In [262]:
df = extract_data(np.arange(2023, 2025, 1), "Q")

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


2023 Bahrain Grand Prix


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '14', '63', '44', '18', '31', '27', '4', '77', '24', '22', '23', '2', '20', '81', '21', '10']
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/877282191.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

2023 Saudi Arabian Grand Prix


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '16', '14', '63', '55', '18', '31', '44', '81', '10', '27', '24', '20', '77', '1', '22', '23', '21', '4', '2']
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/877282191.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '55', '16', '81', '63', '10', '14', '20', '11', '22', '27', '31', '18', '30', '23', '43', '77', '44', '24']
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/877282191.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quali_sessions["GreenLightCount"] = (quali_sessions["Message"].str.contains("GREEN LIGHT")
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/3747660080.py:36: DeprecationWarning: DataFrameGroupBy.apply opera

2024 Mexico City Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '1', '4', '16', '63', '44', '20', '10', '23', '27', '22', '30', '14', '18', '77', '43', '81', '11', '31', '24']
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/877282191.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quali_sessions["GreenLightCount"] = (quali_sessions["Message"].str.contains("GREEN LIGHT")
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/3747660080.py:36: DeprecationWarning: DataFrameGroupBy.apply opera

2024 São Paulo Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '63', '22', '31', '30', '16', '23', '81', '14', '18', '77', '1', '11', '55', '10', '44', '50', '43', '27', '24']
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/877282191.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quali_sessions["GreenLightCount"] = (quali_sessions["Message"].str.contains("GREEN LIGHT")
/var/folders/1p/_mmx_n854bl5ytvfkg4h377m0000gn/T/ipykernel_2776/3747660080.py:36: DeprecationWarning: DataFrameGroupBy.apply opera

In [264]:
df.columns

Index(['Event', 'Driver', 'Team', 'LapTime', 'LapNumber', 'Stint',
       'Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
       'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife',
       'FreshTyre', 'TrackStatus', 'Deleted', 'DeletedReason', 'LapStartDate',
       'QualiSession', 'LapRank', 'CurrentFastestLap', 'DeltaFastestLap'],
      dtype='object')

In [265]:
cols = ["Driver", "QualiSession", "LapStartDate", "LapTime", "LapRank", "CurrentFastestLap", "DeltaFastestLap"]

df[df["Event"]=="2024 United States Grand Prix"][cols].sort_values("LapStartDate").head(10)

,Driver,QualiSession,LapStartDate,LapTime,LapRank,CurrentFastestLap,DeltaFastestLap
273,ZHO,Q1,2024-10-19 22:02:13.812,0 days 00:01:35.479000,1.0,0 days 00:01:35.479000,0 days 00:00:00
106,GAS,Q1,2024-10-19 22:02:22.120,0 days 00:01:35.950000,2.0,0 days 00:01:35.479000,0 days 00:00:00.471000
204,OCO,Q1,2024-10-19 22:02:27.582,0 days 00:01:34.971000,1.0,0 days 00:01:34.971000,0 days 00:00:00
258,BOT,Q1,2024-10-19 22:02:32.564,0 days 00:01:34.714000,1.0,0 days 00:01:34.714000,0 days 00:00:00
126,ALO,Q1,2024-10-19 22:02:36.929,0 days 00:01:34.496000,1.0,0 days 00:01:34.496000,0 days 00:00:00
1,NOR,Q1,2024-10-19 22:03:05.755,0 days 00:01:34.029000,1.0,0 days 00:01:34.029000,0 days 00:00:00
34,SAI,Q1,2024-10-19 22:03:14.636,0 days 00:01:34.104000,2.0,0 days 00:01:34.029000,0 days 00:00:00.075000
53,LEC,Q1,2024-10-19 22:03:24.015,0 days 00:01:34.913000,5.0,0 days 00:01:34.029000,0 days 00:00:00.884000
73,PIA,Q1,2024-10-19 22:03:38.366,0 days 00:01:34.243000,3.0,0 days 00:01:34.029000,0 days 00:00:00.214000
17,VER,Q1,2024-10-19 22:05:52.737,0 days 00:01:33.690000,1.0,0 days 00:01:33.690000,0 days 00:00:00
